## beta_d | beta_t 参数调整

In [39]:
import numpy as np
import matplotlib.plot as plt
import os

In [65]:
BASE_DIR = os.path.abspath('output')
def load_para(dataset):
    paras = []
    BASE_DIR = os.path.abspath('output')
    print BASE_DIR
    file_dir = os.path.join(BASE_DIR, dataset)
    ListDIR = os.listdir(BASE_DIR)
    
    for para in ListDIR:
        if para.find(dataset) > -1:
            if para.find('npz') > -1:
                try:
                    paras.append(np.load(os.path.join(BASE_DIR, para)))
                finally:
                    print 'ok'
            print para
            try:
                paras.append(np.loadtxt(os.path.join(BASE_DIR, para), delimiter=',', encoding='utf-8'))
            finally:
                print "okok"
    return paras
    
# load_para('_e')
dt_name = np.load(os.path.join(BASE_DIR, 'dt_name_e.npz'))
intMat = np.loadtxt(os.path.join(BASE_DIR, 'intMat_e'), delimiter=',')
drugMat = np.loadtxt(os.path.join(BASE_DIR, 'drugMat_e'), delimiter=',')
targetMat = np.loadtxt(os.path.join(BASE_DIR, 'targetMat_e'), delimiter=',')
method = "netlaprls"

In [67]:
def netlaprls_cv_eval(method, dataset, cv_data, intMat, drugMat, targetMat, cvs):
    max_auc, auc_opt = 0, []
    for x in np.logspace(-6, 3, 10):  # [-6, 2]
        for y in np.logspace(-6, 3, 10):  # [-6, 2]
            tic = time.clock()
            model = NetLapRLS(gamma_d=10, gamma_t=10, beta_d=x, beta_t=y)
            cmd = "Dataset:"+dataset+" CVS: "+str(cvs)+"\n"+str(model)
            print cmd
            aupr_vec, auc_vec = train(model, cv_data, X, D, T)
            aupr_avg, aupr_conf = mean_confidence_interval(aupr_vec)
            auc_avg, auc_conf = mean_confidence_interval(auc_vec)
            print "auc:%.6f, aupr: %.6f, auc_conf:%.6f, aupr_conf:%.6f, Time:%.6f\n" % (auc_avg, aupr_avg, auc_conf, aupr_conf, time.clock()-tic)
            if auc_avg > max_auc:
                max_auc = auc_avg
                auc_opt = [cmd, auc_avg, aupr_avg, auc_conf, aupr_conf]
    cmd = "Optimal parameter setting:\n%s\n" % auc_opt[0]
    cmd += "auc: %.6"